In [1]:
import pydwf
import pydwf.utilities
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.signal import butter, lfilter
import datetime
from pathlib import Path

%matplotlib widget

## Open device

In [2]:
## Open device
dwf = pydwf.DwfLibrary()
dev = dwf.deviceControl.open(-1)

## Setup output

In [3]:
## Analog output settings
outCH = 0

## Setup analogOut
anOut = dev.analogOut
node = pydwf.DwfAnalogOutNode.Carrier
anOut.reset(outCH)
anOut.nodeEnableSet(outCH, node, True)
anOut.nodeFunctionSet(outCH, node, pydwf.DwfAnalogOutFunction.Sine)
anOut.nodeAmplitudeSet(outCH, node, 5)

def set_output_freq(freq: float):
    anOut.nodeFrequencySet(outCH, node, freq)

## Setup input

In [4]:
## Analog input settings
inCH = [0, 1]
inFreq = 1e6
inRecordLen = 0.01
inNSamples = int(inFreq * inRecordLen)

## Setup analogIn
anIn = dev.analogIn
anIn.reset()
for CH in inCH:
    anIn.channelEnableSet(CH, True)
anIn.acquisitionModeSet(pydwf.DwfAcquisitionMode.Record)
anIn.frequencySet(inFreq)
anIn.recordLengthSet(inRecordLen)
time.sleep(2)  # Sleep to allow offset to stabilize

## Read data

In [45]:
now = datetime.datetime.now()
for freq in np.linspace(35e3, 65e3, 100):
    ## Start analogOut and analogIn
    set_output_freq(freq)
    anOut.configure(outCH, 1)
    time.sleep(0.1)
    anIn.configure(True, True)

    ## Read data from analogIn
    samples = []
    while True:
        status = anIn.status(True)
        if status in [pydwf.DwfState.Armed, pydwf.DwfState.Ready]:
            continue
        aSamples, _, _ = anIn.statusRecord()

        if aSamples != 0:
            cSamples = np.vstack(
                        [anIn.statusData(idx, aSamples)
                         for idx in inCH]).transpose()
            samples.append(cSamples)

        nSamples = sum(len(x) for x in samples)
        print(f"\r\033[K{freq:10.2f} {100*nSamples/inNSamples: >4.2f}% {aSamples: >5} {nSamples: >9}", end="")

        if status == pydwf.DwfState.Done and nSamples >= inNSamples:
            break

    ## Concatenate all sample chunks
    samples = np.concatenate(samples)

    ## Discard oldest samples
    samples = samples[-inNSamples:]

    out_path = Path("data", "T4P1", f"data_{freq}.csv")
    out_path.parent.mkdir(parents=True, exist_ok=True)
    np.savetxt(out_path, samples, delimiter=",")
    
# Stop output
anOut.configure(-1, 0)
print()

  65000.00 100.02%  2187     10002


## Close device

In [ ]:
dev.close()